In [1]:
import sys
import os
sys.path.append(os.path.abspath("../")) #Add parent directory

In [2]:
# import torch
# import nibabel as nib
# import os
# import numpy as np

## Load data

In [ ]:
from Modular_DL.data_loading import load_nifti, resample_image, convert_to_ras
from Modular_DL.data_normalization import z_score_normalize
import numpy as np
import SimpleITK as sitk

In [ ]:
folder_path = r"C:\Users\User\Desktop\UDG_old_pc\UDG\Subjects\MISSSSSA\Final_project\TrainingValidationTestSets\Training_Set".replace("\\", "/")


output_folder = r"C:\Users\User\Desktop\UDG_old_pc\UDG\Subjects\MISSSSSA\Final_project\TrainingValidationTestSets\Training_resampled".replace("\\", "/")
os.makedirs(output_folder, exist_ok= True)
#target spacing for now: 1,1,1
target_spacing = (1.0, 1.0, 1.0)
reference_shape = None
reference_affine = None
consistent = True

for root, _, files in os.walk(folder_path):
    for file in files:
        if file.endswith(".nii") or file.endswith(".nii.gz"):
            file_path = os.path.join(root, file).replace("\\", "/")

            # Load original image
            data, original_image, voxel_spacing = load_nifti(file_path)

            # Create a SimpleITK image
            image_sitk = sitk.GetImageFromArray(data)
            image_sitk.SetSpacing(voxel_spacing)  # Set the original spacing
            image_sitk.SetDirection(original_image.GetDirection())  # Copy original direction
            image_sitk.SetOrigin(original_image.GetOrigin())  # Copy original origin

            # Resample the image
            is_label = 'seg' in file.lower()  # Determine if the file is a label (segmentation)
            resampled_image = resample_image(image_sitk, target_spacing, is_label=is_label)

            # Convert to RAS orientation
            resampled_image = convert_to_ras(resampled_image)

            # Save the resampled image
            output_file_path = os.path.join(output_folder, f"resampled_{file}")
            sitk.WriteImage(resampled_image, output_file_path)

            print(f"Processed and saved: {output_file_path}")


            # if reference_shape is None:
            #     reference_shape =  resampled_data.shape
            #     reference_affine = affine
            
            # else:

            #     if resampled_data.shape != reference_shape:
            #         print(f"Shape mismatch for file: {file}. Expected {reference_shape}, got {resampled_data.shape}")
            #         consistent = False

            #     if not np.allclose(affine, reference_affine, atol=1e-5):
            #         print(f"Affine mismatch for file: {file}")
            #         consistent = False
            # print("Resampled", resampled_data.shape)


            # print(f"------------------Start {file}--------------------")
            # print(f"File: {file} - Voxel Spacing: {voxel_spacing}")
            # print("\n")
            # print(f"File: {file} - Affine matrix: \n{affine}")
        
            # print(f"------------------End {file}--------------------")
            # print("\n")
            


Resampled (240, 192, 240)
Resampled (240, 192, 240)
Resampled (240, 192, 240)
Resampled (240, 192, 240)
Resampled (240, 192, 240)
Resampled (240, 192, 240)
Resampled (240, 192, 240)
Resampled (240, 192, 240)
Resampled (240, 192, 240)
Resampled (240, 192, 240)
Shape mismatch for file: IBSR_07.nii.gz. Expected (240, 192, 240), got (256, 192, 256)
Affine mismatch for file: IBSR_07.nii.gz
Resampled (256, 192, 256)
Shape mismatch for file: IBSR_07_seg.nii.gz. Expected (240, 192, 240), got (256, 192, 256)
Affine mismatch for file: IBSR_07_seg.nii.gz
Resampled (256, 192, 256)
Shape mismatch for file: IBSR_08.nii.gz. Expected (240, 192, 240), got (256, 192, 256)
Affine mismatch for file: IBSR_08.nii.gz
Resampled (256, 192, 256)
Shape mismatch for file: IBSR_08_seg.nii.gz. Expected (240, 192, 240), got (256, 192, 256)
Affine mismatch for file: IBSR_08_seg.nii.gz
Resampled (256, 192, 256)
Shape mismatch for file: IBSR_09.nii.gz. Expected (240, 192, 240), got (256, 192, 256)
Affine mismatch for f

In [7]:
resampled_data.shape

(214, 192, 214)